This is an interactive lab for learning about energy balance modeling. This physically based approach can be used to model snowmelt volumes, or reconstruct a snowpack (i.e. build the snowpack in reverse). There are a few levels of complexity to the equations. First we will start with the main components and then dig deeper into sucessevie layers of complexity. 

How does snow melt? Snow is frozen water. The first conceptual point for understanding the melt of a snowpack, is the energy required to turn ice into water. The phase change from snow to water requires much more energy than to change the temperature of snow (or water). Positive energy balance will first warm the snow, and then melt a volume of snow, which depends on the amount of energy. 

In [ ]:
#energy avalible to melt assuming mid day during the melt season.
M=280 #Wm-2

#how much snow melts for a constant W m-2 of energy over the course of an hour?
mf=0.0108

melt=M*mf #mm of water
# mm water / (W m^-2 hr), assuming an isothermal (0 deg C) snowpack
print(melt,'mm of water melted in one hour') 

The output above shows how much water would change phase from solid to liquid for a fixed rate of energy into the snowpack over an hour. 

Let's use some real data to figure out how much water is in a snowpack.   
  
First, let's assume a deep snowpack, and we want to figure out how much water melts in the snow pack one day.
[insert image of the conceptual model for the inital EB calculation]
$$M = \text{flux of energy melting snow pack} $$  
  
  First we have to understand the net radiation being absorbed or emitted from the surface.  
  Radiation comes from the following sources:  
$$S=\text{Shortwave radiation from sun}$$
$$\alpha=\text{snowpack albedo, how much solar radiation gets reflected}$$  
$$L_{down}=\text{longwave radiation from atmosphere}$$    
$$L_{up}=\text{emission from surface (based on temperature of the surface and emissivity)}$$    
$$R_{net}=S(1-\alpha)+L_{down}-L_{up}$$   

We use $S(1-\alpha)$ to caclulate the amount of solar radiation absorbed by the snowpack becuase we do not diretly measure this quantity $S$ and $\alpha$ are easier to measure.  
  

  
$$R_{net} = \text{net radiation (shortwave and longwave), the flux of energy into or out of the snowpack}$$ 


$$H =  \text{sensible heat flux, i.e. raising the temperature of water in a kettle}$$   
  
$$S = \text{latent heat flux, i.e. boiling water in a kettle (phase change of the water from liquid to vapor)}$$  
  
$$G = \text{heat flow into or out of the pack, we assume this is negligible compared to the other fluxes}$$  


Given these fluxes, we have the following equation for the energy that is avalible to melt the snowpack:  
  
$$M = R_{net} + H + L + G   \text{ and } G = 0$$  
  
$$\text{so}$$   
  
$$M = R_{net} + H + L$$  
  
for each timestep. And the snowpack only melts when there is energy (M) available to melt (otherwise it cools), so   
  
$$ M = max( R_{net} + H + L, 0 )$$

Lets now go look at some real data for all the parts of these quesitons and see how meausrements of the snowpack and the energy balance can be used to estimate SWE


We will use 2013 to 2017 Cold Regions Research and Engineering Laboratory and University of California Santa Barbara (CUES) data to explore. More information on CUES can be found here:
https://vesr.nrs.ucsb.edu/about/facilities/snarl/cues-snow-science-laboratory
https://snow.ucsb.edu/

In [ ]:
#we will use the pandas, numpy, and matplotlib pyplot libraires in this tutorial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import log as ln


#read in 6 years of energy balance data from our field station
# tis code might clean up the data on the backend with out having to have steps for it
# read_csv('CUES_2013_to_2017_EBdata.csv', index_col=0, parse_dates=True)
EBdata=pd.read_csv("CUES_2013_to_2017_EBdata.csv")
EBdata['DateTime']=pd.to_datetime(EBdata['DateTime'],format='%m/%d/%Y %H:%M')
EBdata=EBdata.set_index('DateTime')
EBdata.head()

Let's look at the dataset we just imported as a table to see what energy balance variables are included.

This shows us all the types of data we have imported and the number of records - 60865. Let's convert the first column into a datatype that recognizes these values as specific dates and times and then set these timestamps as the index for the dataframe. Timbo: graph below should be plotte

In [ ]:
EBdata.plot(y=['SnowDepth_cm','SWE_cm'],figsize=(15,4))

We can see each winter season at the point as the snow depth and the snow water equivalent increase each year and then the snow melts out each spring. There is no SWE data for the first two seasons as the instrument that measures SWE was not installed. In this lab, we will use the snow depth measurement as a snow cover identifer (snow on / snow off) along with the energy balance data collected at this site to reconstruct the snow water equivalent from the energy balance each day. We will then compare our energy balance estimate to the independent SWE measurement at the site

lets look at the solar radiation, the incoming solar radiation and the reflected solar radiation off the snow. The amount of reflected solar radiation is equal to the incoming solar radation that is relfectied - i.e. incoming*albedo

In [ ]:
EBdata['reflected_solar']=EBdata['uplooking_broadband_watts_per_meterxx2']*EBdata['direct_broadband_snow_albedo']
EBdata['reflected_solar']=EBdata['reflected_solar'].fillna(0)

Lets look at one week of data in may of 2017 durring the melt season

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['uplooking_broadband_watts_per_meterxx2','reflected_solar'],figsize=(15,4))

Can you guess which days were cloudy? which days were sunny?

The net solar flux is S(1-albedo)  
is incoming solar - reflected solar. This is the net flux of solar energy into the snowpack each day

In [ ]:
EBdata['net_solar']=EBdata['uplooking_broadband_watts_per_meterxx2']-EBdata['reflected_solar']
EBdata.loc['5/1/2017':'5/7/2017'].plot(y='net_solar',figsize=(15,4))

Lets look at the longwave energy balance. 

We do not have snow surface temperature measurements, but we can approximate,  we know when there is snow cover, from the depth meausrement, and we know that snow cannot be warmer than 0degC. So Tsnow = min(Tair,0) if snow_depth>0
# Use dewpoint instead of air temp, see Raleigh et al 2013 http://dx.doi.org/10.1002/2013WR013958


The longwave radiation of a object is given by the Stefan Boltzman equation and is a function of the object's temperature. (hotter objects emit more radiation)

[insert good photo here]  

emissivity.ems=single(0.99);% Emissivity of snow

In [ ]:
#snow is almost a black body emitter
snow_emissivity=0.99
celcius_2_kelvin=273.15
#stefan-boltzman constant
sbc=5.67e-8
#want the temperature in kelvin for longwave emission calculation
EBdata['snow_surface_temp']=np.minimum(0,EBdata['AirTemp_C'])
EBdata.loc[EBdata['SnowDepth_cm']==0,'snow_surface_temp']=np.nan
EBdata['outgoing_longwave']=sbc*((EBdata['snow_surface_temp']+celcius_2_kelvin)**4)

Let look at all the energy balance variables over this time period.  

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['uplooking_longwave_watts_per_meterxx2','outgoing_longwave'],figsize=(15,4))

In [ ]:
EBdata['net_longwave']=EBdata['uplooking_longwave_watts_per_meterxx2']-EBdata['outgoing_longwave']
EBdata.loc['5/1/2017':'5/7/2017'].plot(y='net_longwave',figsize=(15,4))

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['net_solar','net_longwave'],figsize=(15,4))

sensible heat flux = density of air * specific heat of air at constant pressure * (Tair - Tsurface) / aerodynamic resistance over snow adjusted for instability  
$$H=\rho_aC_pD\zeta(T_a-T_s)$$

In [ ]:
#constants needed to calculate heat exchanges
Cp=1010 # #specific_heat_of_air J/(kg deg)
k=0.41#Von Karman's constant
eta=9.4 #stability function constant
phi=5.3 #stability function constant
g=9.8 # gravity m/s2
celcius_2_kelvin=273.15 #temperatures must be in kelvin
kPa_0=101.325; # sea level pressure kPa.
rho_0=1.29 # sea level rho
z0=0.0005# m roughness height (0.5mm)

#variables needed to perform heat exchange calculations
Ts=EBdata['snow_surface_temp']+celcius_2_kelvin
Ta=EBdata['AirTemp_C']+celcius_2_kelvin
u=EBdata['wind_speed_meters_per_sec'] #m/s
zR=(600-EBdata['SnowDepth_cm'])/100 # meters height of windspeed measurement 6m above bare ground - correct for snow depth
kPa_alt=EBdata['AirPressure_mb']/10 #measured pressure at snow study site, convert mb to kPa
#air_density (rho) alt adj. density air,kg/m^3
rho_air=rho_0*(kPa_alt/kPa_0);


#heat_exchange_coefficent (also used for latent heat flux calculation)
D=(u*k**2)/((ln(zR/z0))**2)

#nondimensional stability function, three situations (neutral,stable,unstable) (also used for latent heat flux calculation)
#unstable(Ts>Ta)
#gamma=phi*eta*(k/(ln(zR/z0)))**2*np.sqrt(zR/z0)
C1=phi*eta*(k/(ln(zR/z0)))**2*np.sqrt(zR/z0)

#Ri=(g*zR*(Ta-Ts))/(Ta*u**2)
C2=g*zR/(Ta*u**2)

B1=eta*C2
B2=C1*np.sqrt(np.absolute(C2))
B3=1+B2*np.sqrt(np.absolute(Ts-Ta))

#unstableConditions=1-(eta*Ri)/(1+gamma*np.sqrt(np.absolute(Ri)))
unstable=1+B1*(Ts-Ta)/B3

#stable(Ts<Ta)
#stableConditions=1+(eta/2)*Ri
B8=B1/2
stable=1/((1+B8*(Ta-Ts))**2)

#neutral stability assumption first (Ts==Ta) 
stability=np.ones_like(unstable)

#fill in unstable times (Ts>Ta)
unstableIdx=Ts>Ta
stability[unstableIdx]=unstable[unstableIdx]

#fill in stable times (Ts<Ta)
stableIdx=Ts<Ta
stability[stableIdx]=stable[stableIdx]

#add sensible heat flux as a new column to the dataframe
EBdata['Sensible_Heat_Flux']=rho_air*Cp*D*stability*(Ta-Ts)

lets plot some of the variables by themselves and make sure they look right - plug various variables into the plotting function beblow to see what they look like across the season

In [ ]:
plt.plot(zR) #change the variable to plot here (replace zR with others) to check and see what they look like

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['Sensible_Heat_Flux'],figsize=(15,4))

latent heat flux = latent heat of vaporization or sublimation* density of air * (0.622/air pressure) * (vapor pressure of air - saturated vapor pressure of snow surface)  
$$L=\rho_a\lambda_{v,s}D\zeta(\theta\frac{e_a-e_s}{P_z})$$

In [ ]:
#information from sensible heat flux calcualtion we need for latent heat flux calculation
Ts=Ts
Ta=Ta
rho_air=rho_air
Pa_alt=kPa_alt*1000
D=D

#new information needed for latent heat flux calculation
theta=0.622 # ratio of molecular weight of water to air
latent_heat_of_sublimation=2834700 #J/kg
latent_heat_of_vaporization=2501000 #J/kg
RH=EBdata['RH']
Ta_C=EBdata['AirTemp_C']

#vapor pressure of the air
#calculated based on the saturation vapor pressure of the dewpoint temperature 
# as in: (Raleigh et al (2013) doi:10.1002/2013WR013958)
c=273.86
b=22.587
#celcius dewpoint temp
TdC=c*(ln(RH)+b*Ta_C/(c+Ta_C))/(b-ln(RH)-b*Ta_C/(c+Ta_C))
#dewpoint temp (K)
Td=TdC+273.15
T0=273.16
es0=0.611 #saturation vapor pressure in kPa at 273.16K
Lwater=6808*(1/T0-1/Td)-5.09*ln(Td/T0)
vp=es0*np.exp(Lwater)
#saturation vapor pressure of ice at dewpoint temp (if air temp is below 0C)
Lice=6293*(1/T0-1/Td)-0.555*ln(Td/T0)
resetVp=Td<=T0
#Lice[Lice!=resetVp]=np.nan
vp_ice=es0*np.exp(Lice)
vp[resetVp]=vp_ice[resetVp]
eA=vp*1000 #eA and eS are in Pa, not kPa


#saturation vapor pressure at the snow surface (vapor pressure over ice from Buck 1981) (e at the snow surface)
Tf=273.15
A=6.1115*100
B=22.452
C=272.55
eS=A*np.exp((B*(Ts-Tf))/(C+(Ts-Tf)))
#enthalpy of vaporization or sublimation 
#is liquid water in the snow evaporating or ice sublimating?
#vapor pressure is greater at the snow surface - we assume sublimation - (use enthalpy of sublimation) 
#or greater in the air above the snow - we assume vaporization - (use enthalpy of vaporization)
xLs=np.full_like(eS,latent_heat_of_sublimation) #set all to sublimation
xLs[eS<eA]=latent_heat_of_vaporization #set to vaporization if nessesary

EBdata['Latent_Heat_Flux']=rho_air*xLs*D*stability*(theta*((eA-eS)/Pa_alt)) # no adjustments for vegetation emitted longwave

In [ ]:
plt.plot(Td) #change the variable to plot here (replace Td with others) to check and see what they look like

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['Latent_Heat_Flux'],figsize=(15,4))

Lets look at each componenet of the energy balance for this week. We can take the individual coponents and look at the net flux of energy into or out of the snowpack at each hourly timestep

In [ ]:
EBdata['Net_Flux']=EBdata['Latent_Heat_Flux']+EBdata['Sensible_Heat_Flux']+EBdata['net_solar']+EBdata['net_longwave']
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['Net_Flux','Latent_Heat_Flux','Sensible_Heat_Flux','net_solar','net_longwave'],figsize=(15,4))

Solve the energy balance each day for potiental melt
$$M_p=max(R_{net}+H+L,0)$$

In [ ]:
EBdata['Energy_to_Melt']=np.maximum(EBdata['Net_Flux'],0)

In [ ]:
EBdata.loc['5/1/2017':'5/7/2017'].plot(y=['Energy_to_Melt'],figsize=(15,4))

Reconstruction works backwards up to peak SWE, lets figure out the timeperiod to run our model based on the SWE measurements at CUES. We will figure out the dates to run reconstruction based on peak SWE date to melt out date. Because the reconstruciton method is independent from the SWE pillow measurements, we can compare the two results at the end

In [ ]:
#lets focus on the 2017 water year
WY2017=EBdata.loc['10/1/2016':'9/30/2017']
#date of peak SWE and volumne of water in the snowpack that day
maxSWE=WY2017['SWE_cm'].max()
maxSWEdate=WY2017['SWE_cm'].idxmax()

#melt out date of the snowpack (hacky solution right now)
meltOutIdx=WY2017['SWE_cm']==0                    
lastSWEdate=max(meltOutIdx[meltOutIdx.diff() != 0 ].index.tolist())
#.where(WY2017['SWE_cm'][:-1] != WY2017['SWE_cm'][1:],0)

                    
                    
WY2017.plot(y=['SWE_cm'],figsize=(15,4))
plt.scatter(maxSWEdate,maxSWE,marker='x',c='r',s=200)
plt.scatter(lastSWEdate,0,marker='x',c='r',s=200)

print(maxSWE, 'cm is the maximum SWE of the snowpack')
print(maxSWEdate , 'is the max SWE date')
print(lastSWEdate,' is the melt out date')
print(np.count_nonzero(meltOutIdx))

Now lets use our energy balance information to backcalculate the SWE each day from melt out date to peak SWE date and then compare our energy balance SWE estimate to the snow pillow data

In [ ]:
#valid time steps for summing melt over
meltPeriod=EBdata[maxSWEdate:lastSWEdate]
#how much snow melts for a constant W m-2 of energy over the course of an hour?
melt_factor=0.0108  # mm/((W/m2)h)
melt_per_hour=(meltPeriod['Energy_to_Melt']*melt_factor)/10 #mm to cm
#cumulative sum of the melt_per_hour from melt out date back to peak SWE
SWE=melt_per_hour.loc[::-1].cumsum()[::-1]
SWE.plot(y=['SWE'],figsize=(5,5),label="SWE Reconstruction")
melt_per_hour.plot(y=['melt_per_hour'],label="melt per hour")
plt.legend(loc="upper right")

As you can see from above, a little melt each hour adds up to a lot of snow.
  
    
Lets now look at our reconstructed snow water equivalnet compared to the measured SWE at the snow pillow.
#and of course it matches nearly perfectly :)

In [ ]:
WY2017.plot(y=['SWE_cm'],figsize=(15,5), label=['SWE pillow measurement'])
SWE.plot(y=['SWE'],label="SWE Reconstruction")
plt.scatter(maxSWEdate,maxSWE,marker='x',c='r',s=200)
plt.scatter(lastSWEdate,0,marker='x',c='r',s=200)
plt.legend(loc="upper right")